# Assignment 5

## Language models


Author: Thomas Vogenthaler.

In this assignment we will explore language models.
To do so, we will encapsulate a language model using a class called `language_model` which implements language models with Laplace add-one smoothing.
An overview of the methods your class should have is as follows:

In [1]:
import string
import itertools
import re
import decimal
!pip install pandas
##Bash script to import pandas
import pandas as pd 
import glob
from collections import defaultdict
from math import log2
#from string import maketrans
class language_model:

    def __init__(self, ngram=1) :
        """
        Initialize a language model
        """
        self.V = -1
        ##Set V = -1 this is for our ngram usage
        self.ngram = ngram
        #Declare ngram within self
        self.sentences =[]
        #This is so we can split up into sentence
        self.model = defaultdict(lambda: 0)
        #This is our model that we are using
        self.modelminus = defaultdict(lambda: 0)
        #Model minus is set to defauldict lambda function
        self.words = 0
        self.ngramCount = -1
        self.ngramE = -1
#################################################################################################################
    def train(self, file_name) :
        """
        train a language model
    
        Parameters:
        file_name is a file that contains the training set for the model
        """
        """
        Open file and set text equal to the content over the file.
        This is so we can read in the file
        Be able to parse the data into unigram, Bigram, and trigram
        
        
        After its parsed we can use sparcity and perplexity 
        
        """
        
        self.V = self.unigrams(file_name)
        self.ngramE = self.V ** self.ngram
        ##Set the self = to the clean text
        input_text = "?:!-\n"
        output_string = "...  "
        """Have to covert all the strings.
        We have to remove all the unecessary characters from the text
        To do this we have parse out all the text"""
        remove_char = ",;\"(\)\\\/"
        begin_char= "<s> "
        end_char = " </s>"
        """Remove the strings of / \ and others
        Get the beginning character
        Get the end end character
        Check for those characters 
        Next we have to convert"""
        trans = str.maketrans(input_text ,output_string ,remove_char)
        store = defaultdict(lambda: 0)
        """Here we convert all the string"""
        storeminus1 = defaultdict(lambda: 0)
        self.V = self.unigrams(file_name)
        V = defaultdict(lambda: 0)
        with open(file_name) as fp:
            line = fp.readline()
            #line =" "
            while line:
                line = fp.readline()
                line_holder = line.lower()
                line_holder = line_holder.translate(trans)
                line_holder = line_holder.split('. ')
                """T is going to be our iterator through the line holder"""
                for T in line_holder:
                    """This for loop will start to parse the text into sentences
                    Check error for strip. 
                    Getting lower end?
                    Not sure check with TA."""
                    T = T.strip()
                    Q = begin_char+T+end_char
                    Q = Q.strip()
                    lengt = len(Q.split(' '))
                    total_amount = self.ngram
                    if(total_amount == 1):
                        for l in range(1,lengt):
                            store[Q.split(' ')[l]] = store[Q.split(' ')[l]]+1
                    if(total_amount > 1):
                        """This will split the words into more sentences."""
                        for r in range(0,lengt-total_amount+1):
                            line_holder = tuple(Q.split(' ')[r:r+(total_amount)])
                            store[line_holder] = store[line_holder]+1 
                            line_holder = tuple(Q.split(' ')[r:r+(total_amount-1)])
                            storeminus1[line_holder] = storeminus1[line_holder]+1
        self.model = store
        self.modelminus = storeminus1
        self.ngramE = self.V ** self.ngram
        self.ngramCount = len(store)
        if(self.ngram > 1):
            self.ngramCount = len(storeminus1)
        print("Created {} ngrams out of {} potentetial, with a sparsity of: {}\n".format(self.ngramCount,self.ngramE,(self.ngramE - self.ngramCount)/self.ngramE))
        pass
#################################################################################################################
    def unigrams(self, file_name):
        input_text = "?:!-\n"
        output_string  = "...  "
        """Unigrams will be calculated for the entire word bank
        
        Essentially all the words that are singles are going to be calculated for
        the entire unigram
        """
        remove_char  = ",;\"(\)\\\/"
        begin_char = "<s> "
        end_char = " </s>"
        trans = str.maketrans(input_text,output_string ,remove_char )
        store = defaultdict(lambda: 0)
        with open(file_name) as fp:
            line = fp.readline()
            while line:
                line = fp.readline()
                line_holder= line.lower()
                line_holder= line_holder.translate(trans)
                line_holder = line_holder.split('. ')
                for T in line_holder:
                    T = T.strip()
                    Q = begin_char+T+end_char
                    Q = Q.strip()
                    lengt = len(Q.split(' '))
                    total_amount = self.ngram
                    for l in range(1,lengt):
                        ##Check with TA here??? Broken
                        store[Q.split(' ')[l]] = store[Q.split(' ')[l]]+1
                
        return len(store)
#################################################################################################################
    def content(self, file):
        """Content will take the the input string we need to parse out
        After w eparse out the unecessary character which include
        ? : - \ m
        as well as ...
        Then remove all the other slashes.
        
        Eseentially the content we need to remove."""
        input_text = "?:!-\n"
        output_string  = "...  "
        remove_char  = ",;\"(\)\\\/"
        trans = input_text.maketrans(input_text ,output_string ,remove_char)
        return file.read().lower().translate(trans)
#################################################################################################################    
    def clean(self, file):
        sent = file.split('.')
        sentences = []
        """Clean up file by splitting into sentences
        Afer we split up the and it is cleaned into sentences
        This is what our clean function will do
        """
        for S in sent:
            sentence = ''
            for W in S.strip().split(' '):
                if re.match(r'\S',W.strip()):
                    sentence += W.strip()+ ' '
            sentences.append("<s>" + sentence + "</s>")
            
        return tuple(sentences)
#################################################################################################################
    def grammer_split(self,sentences,n):
        G = []
        """Grammar split will split it into stences
        so one that we can run all our functions on it
        If we do not do this. we will get errors.
        """
        for k in sentences:
            W = k.split(' ')
            for i in range(len(W) - n + 1):
                Gn = ''
                for j in range(n):
                    Gn += W[i+j] + ' '
                G.append(Gn.strip())
                
        return tuple(G)
#################################################################################################################
    def perplexity(self, ngrams):
        N = len(ngrams)
        perplexity = 0
        """This function will calculate the preplexity of the text
        
        in which then w ewill loop through gram in ngrams
        Doing a strip the entire time"""            
        for gram in ngrams:
            pro = ''
            if self.ngram > 1:
                for i in range(0, self.ngram-1):
                    pro += gram.split(" ")[i] + " "
                    
            pro = pro.strip()
            countN = 0
            if gram in self.model:
                countN = self.model[gram]
                    
            countminusN = 0
            if pro in self.modelminus:
                countminusN = self.modelminus[pro]
            perplexity += log2((countN + 1) / (countminusN + self.V))
        perplexity = 2 ** ((-1/N) * perplexity)
        return perplexity   
#################################################################################################################
    def test(self, file_name) :
        input_text = "?:!-\n"
        output_string  = "...  "
        remove_char = ",;\"(\)\\\/"
        begin_char = "<s> "
        end_char= " </s>"
        """Just as same as the other implementations
        we have to remove the unecessary character
        begining char
        end char
        and being able to transform it 
        
        Using the input text which will then be outputted to the output_string"""
        trans = str.maketrans(input_text,output_string ,remove_char )
        store = defaultdict(lambda: 0)
        storeminus1 = defaultdict(lambda: 0)
        self.V = self.unigrams(file_name)
        V = defaultdict(lambda: 0)
        with open(file_name) as file_picker:
            line = file_picker.readline()
            #line =" "
            while line:
                line = file_picker.readline()
                line_holder = line.lower()
                line_holder = line_holder.translate(trans)
                line_holder = line_holder.split('. ')
                for T in line_holder:
                    T = T.strip()
                    Q = begin_char+T+end_char
                    Q = Q.strip()
                    lengt = len(Q.split(' '))
                    total_amount = self.ngram
                    if(total_amount == 1):
                        for l in range(1,lengt):
                            store[Q.split(' ')[l]] = store[Q.split(' ')[l]]+1

                    if(total_amount > 1):
                        for r in range(0,lengt-total_amount+1):
                            line_holder = tuple(Q.split(' ')[r:r+(total_amount)])
                            store[line_holder] = store[line_holder]+1 
                            line_holder = tuple(Q.split(' ')[r:r+(total_amount-1)])
                            storeminus1[line_holder] = storeminus1[line_holder]+1

        self.model = store
        self.modelminus = storeminus1
        """Text is set to self content
        OPen file
        Conduct grammer commit and then we will be able to text perplexity
        IF perplexity values are low then we are doing something incorrect.
        """
        text = self.content(open(file_name,'r'))
        sentances = self.clean(text)
        test = self.grammer_split(sentances,self.ngram)
        perplexity = self.perplexity(test)
        return perplexity    

In [2]:
tests = {
    """tests unigram
    Bigram
    and trigram
    using language model.
    Total using ngrams as well"""
    "Unigram": language_model(1),
    "Bigram": language_model(2),
    "Trigram": language_model(3)
}
books = ["pride_and_prejudice.txt","sense_and_sensibility.txt","persuasion.txt","jane_eyre.txt"]
preplexingresults = {gram: {} for gram in tests.keys()}
"""
Testing for the ngram in which we will call it total_gram

loops will be used in test keys
we will be collecting all the keys
printing out to panda format"""
for total_gram in tests.keys():
    tests[total_gram].train(books[0])
    for j in range(len(books)):
        preplexingresults[total_gram].update({books[j]: tests[total_gram].test(books[j])})
for total_gram in tests.keys():
    print(total_gram)
    for text in preplexingresults[total_gram].keys():
        print('Perplexity of {}: {}'.format(text,preplexingresults[total_gram][text]))
        """
        Printing out in panda format here."""
col = tests.keys()
index = books
values = preplexingresults
pd.DataFrame(preplexingresults,index = index,columns = col)

Created 6464 ngrams out of 6464 potentetial, with a sparsity of: 0.0

Created 6464 ngrams out of 41783296 potentetial, with a sparsity of: 0.9998452970297029

Created 49649 ngrams out of 270087225344 potentetial, with a sparsity of: 0.999999816174201

tests unigram
    Bigram
    and trigram
    using language model.
    Total using ngrams as wellUnigram
Perplexity of pride_and_prejudice.txt: 30.314312620957555
Perplexity of sense_and_sensibility.txt: 30.6189037810618
Perplexity of persuasion.txt: 39.12732957122242
Perplexity of jane_eyre.txt: 56.890541415807775
Bigram
Perplexity of pride_and_prejudice.txt: 6463.999999984141
Perplexity of sense_and_sensibility.txt: 6488.000000069661
Perplexity of persuasion.txt: 5914.999999926937
Perplexity of jane_eyre.txt: 12960.000000167925
Trigram
Perplexity of pride_and_prejudice.txt: 6463.999999998228
Perplexity of sense_and_sensibility.txt: 6488.000000054027
Perplexity of persuasion.txt: 5914.999999931532
Perplexity of jane_eyre.txt: 12960.00000

,tests unigram Bigram and trigram using language model. Total using ngrams as wellUnigram,Bigram,Trigram
pride_and_prejudice.txt,30.314313,6464.0,6464.0
sense_and_sensibility.txt,30.618904,6488.0,6488.0
persuasion.txt,39.127330,5915.0,5915.0
jane_eyre.txt,56.890541,12960.0,12960.0


### Your tasks:

Complete the above class for training and evaluating
language models using unigrams, bigrams, and trigrams.
Before using a text for training or testing:
* Convert the text to lower case.
* Convert question marks (?), colons (:) and exclamation marks (!) to periods.
* Remove all punctuation marks other than the period (commas, semicolons, underscores and quotes); dashes should be converted to spaces.
* Parse the text into sentences, adding beginning of sentence and end of sentence tokens.

For performing the transformations of the text use Python's string method `str.translate`, which also uses a conversion table that is created by the string method `str.maketrans`.
Finally for storing your language model, use a Python dictionary.  You may find it useful to use Python's `defaultdict`, which is part of the `collections` module.



Using the code you have written, demonstrate how you would use language models to distinguish between the writings of Jane Austen and some of her contemporaries.  We have provided the text for the following books for you to use in ASCII format:
* Pride and Prejudice by Jane Austen
* Persuasion by Jane Austen
* Sense and Sensibility by Jane Austen
* Jane Eyre by Charlotte Bronte
* Alice in Wonderland by Lewis Carroll

These can be downloaded [here](https://www.cs.colostate.edu/~cs440/fall19/assignments/texts.tar.gz).
Original versions of the text are from the [Project Gutenberg](http://www.gutenberg.org/) website. 

In your analysis, compare the ability of unigram, bigram and trigram models for this task.  Explain your observations.


Finally, analyze the models you have constructed for Pride and Prejudice.  How sparse are the bigram and trigram models?  I.e., what fraction of the total number of entries in the model before smoothing are zero?


###ANALYSIS REPORT.

Sparsity for sparisty when we had to calculate it for unigram. We didn't see any signicant result, this is due to the fact that the vocabulary and unigram had the same integer value. Overall for sparsity, unigrams did not have a full result to compare and contrast within natrual language processing
Bigrams, the sparisty for bigrams always returned a value that was always usually infinitely closer to one
sparsity for the trigrams however were always closer to one. this is because the bigram model works better than not only unigram but trigram as well
Trigrams has the highest sparsity total, which means its the best model to use out of all three of them. But if the ngram were larger, than time complexity would be much larger which is also a factor to consider
    
Perplexity shows that we have changes in text as we grow in preplexity, which shows us that the more text we have of the same author and comparing it to other books written by the same author, tends ot have a high preplexity. Calculating the preplexity showed us that unigram had the smallest value, bigrams had the largest preplexity change increase, and the preplexity calculated for trigrams shows that there wasn't a great increase. (Look at data above)

    
    

## Submission

Answer the questions in the cells reserved for that purpose.
Submit your report as a Jupyter notebook via Canvas. Running the notebook should generate all the results in your notebook.  Leave the output of your notebook intact so we don't have to run it to see the results.

## Grading

Grading will be based on the following criteria:

* Your code solves the specified problem.
* You performed an analysis that addresses the problem.
* Overall readability and organization of the notebook.
* Effort in making interesting observations where required.
* Conciseness. Points may be taken off if the notebook is overly long.